# Libraries

In [2]:
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
%%R
version$version.string

# install.packages("tidyverse")

# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("ggtree")
# install.packages("ggstance")

# install.packages("Cairo")
# install.packages("showtext")
# install.packages("stringr")

#install.packages("stringi", type = "source")


[1] "R version 4.2.2 (2022-10-31)"


In [3]:
%%R
library(tidyverse)
library(ggtree)
library(ggstance)
library(Cairo)
library(showtext)
library(stringr)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Registered S3 methods overwritten by 'treeio':
  method              from    
  MRCA.phylo          tidytree
  MRCA.treedata       tidytree
  Nnode.treedata      tidytree
  Ntip.treedata       tidytree
  ancestor.phylo      tidytree
  ancestor.treedata   tidytree
  child.phylo         tidytree
  child.treedata      tidytree
  full_join.phylo     tidytree
  full_join.treedata  tidytree
  groupClade.phylo    tidytree
  groupClade.treedata tidytree
  groupOTU.phylo      tidytree
  groupOTU.treedata   tidytree
  is.rooted.treedata  tidytree
  nodeid.phylo        tidytree
  nodeid.treedata     tidytree
  nodelab.phylo       tidytree
  nodelab.treedata    tidytree
  offspring.phylo     tidytree
  offspring.treedata  tidytree
  parent.phylo        tidytree
  parent.treedata     tidytree
  root.treedata       tidytree
  rootnode.phylo      tidytree
  sibling.phylo       tidytree
ggtree v3.6.2 For help: https://yulab-smu.top/treedata-book/

If you use the ggtree package suite in published resea

In [4]:
%%R
genome_data <- read_csv("data/Enterobacteriaciae_species_merged_1000.csv")
genome_data


Rows: 8287 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): accession, ncbi_organism_name, ncbi_taxonomy, gtdb_taxonomy
lgl (2): gtdb_species_representative, ncbi_type_material

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 8,287 × 6
   accession       ncbi_organism_name                ncbi_taxonomy gtdb_taxonomy
   <chr>           <chr>                             <chr>         <chr>        
 1 GCA_002294575.1 Enterobacteriaceae bacterium UBA… d__Bacteria;… d__Bacteria;…
 2 GCA_002307955.1 Enterobacteriaceae bacterium UBA… d__Bacteria;… d__Bacteria;…
 3 GCA_002317605.1 Enterobacteriaceae bacterium UBA… d__Bacteria;… d__Bacteria;…
 4 GCA_002319825.1 Enterobacteriaceae bacterium UBA… d__Bacteria;… d__Bacteria;…
 5 GCA_002333165.1 Enterobacteriaceae bacterium UBA… d__Bacteria;… d__Bacteria;…
 6 GCA_00233

In [5]:
%%R
genomes_downloaded <- read_csv("data/downloaded_genomes_list.txt", col_names = FALSE)
length(genomes_downloaded)
head(genomes_downloaded)


Rows: 7981 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): X1



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 1
  X1             
  <chr>          
1 GCA_000168075.1
2 GCA_000214745.2
3 GCA_000240805.2
4 GCA_000250435.2
5 GCA_000264705.1
6 GCA_000268105.1


In [6]:
%%R
#select genomes in the genomes_downloaded list
genome_data <- genome_data[genome_data$accession %in% genomes_downloaded$X1,]
length(genome_data$accession)


[1] 7981


In [8]:
%%R
hmm_hits <- read.csv("data/final_output.csv")
# head(hmm_hits)


In [9]:
%%R

# Make 'genome' column from 'input_file' column, remove text after second '_' character
hmm_hits$genome <- gsub("^([^_]*_[^_]*).*", "\\1", hmm_hits$input_file)

#print
head(hmm_hits$genome, 20)


 [1] "GCF_013886195.1" "GCF_013886335.1" "GCA_904866245.1" "GCF_000354355.2"
 [5] "GCF_000356565.1" "GCF_002971585.1" "GCF_002972375.1" "GCF_002972935.1"
 [9] "GCF_002972995.1" "GCF_002310575.1" "GCF_001263065.1" "GCF_002190675.1"
[13] "GCF_002475215.1" "GCF_002965825.1" "GCF_004960225.1" "GCF_005144925.1"
[17] "GCF_005383645.1" "GCF_013834525.1" "GCF_016504085.1" "GCF_018442805.1"


In [10]:
%%R

# Make new blank column in hmm_hits dataframe
hmm_hits$gtdb_taxonomy <- NA

# Match the first 13 characters of the contig column between hmm_hits and all_best_hits
for (i in 1:nrow(hmm_hits)) {
    hmm_hits$gtdb_taxonomy[i] <- genome_data$gtdb_taxonomy[which(genome_data$accession == hmm_hits$genome[i])]
}

# Print the updated dataframe1
head(hmm_hits$gtdb_taxonomy)


[1] "d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia; s__Escherichia fergusonii"
[2] "d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia; s__Escherichia fergusonii"
[3] "d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia; s__Escherichia marmotae"  
[4] "d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia; s__Escherichia coli"      
[5] "d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia; s__Escherichia coli"      
[6] "d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia; s__Escherichia coli"      


In [11]:
%%R
Enterobacteriaciae_species_tree <- read.tree("data/Enterobacteriaciae_species.tree")
Enterobacteriaciae_species_tree$tip.label

 [1] "Citrobacter_werkmanii"      "Citrobacter_youngae"       
 [3] "Citrobacter_braakii"        "Citrobacter_koseri"        
 [5] "Citrobacter_amalonaticus"   "Salmonella_bongori"        
 [7] "Salmonella_enterica"        "Escherichia_fergusonii"    
 [9] "Escherichia_albertii"       "Escherichia_marmotae"      
[11] "Escherichia_coli"           "Klebsiella_quasipneumoniae"
[13] "Klebsiella_pneumoniae"      "Klebsiella_variicola"      
[15] "Klebsiella_aerogenes"       "Klebsiella_oxytoca"        
[17] "Klebsiella_michiganensis"   "Raoultella_ornithinolytica"
[19] "Raoultella_planticola"      "Pluralibacter_gergoviae"   
[21] "Enterobacter_cancerogenus"  "Enterobacter_asburiae"     
[23] "Lelliottia_nimipressuralis" "Enterobacter_kobei"        
[25] "Enterobacter_cloacae"       "Enterobacter_ludwigii"     
[27] "Enterobacter_mori"          "Enterobacter_hormaechei"   
[29] "Lelliottia_amnigena"        "Leclercia_adecarboxylata"  
[31] "Kosakonia_cowanii"          "Kosakonia_sacchari" 

In [12]:
%%R
samples <- read.csv("data/Enterobacteriaciae_accessions.csv", header = TRUE)
samples <- samples[samples$accession %in% genomes_downloaded$X1,]
length(samples$accession)
#head(samples)

[1] 7981


In [13]:
%%R
samples %>%
	separate(gtdb_taxonomy, c('f1','f2','f3','f4','f5','f6','f7','f8'), sep = "__") %>%
	count(f8) %>%
	#remove NA
	na.omit()

                                   f8   n
1                Actinomyces marmotae   5
2           Anaerotruncus colihominis   1
3             Atlantibacter hermannii  28
4              Brachyspira pilosicoli   1
5                Campylobacter_D coli  33
6                 Citrobacter braakii 119
7               Citrobacter werkmanii  65
8                 Citrobacter youngae  40
9          Citrobacter_A amalonaticus  57
10       Citrobacter_A amalonaticus_C   1
11       Citrobacter_A amalonaticus_E   2
12               Citrobacter_B koseri 108
13       Citrobacter_C amalonaticus_A   4
14             Cronobacter condimenti   3
15            Cronobacter dublinensis  42
16            Cronobacter malonaticus  62
17             Cronobacter muytjensii   8
18              Cronobacter sakazakii 436
19            Cronobacter universalis   4
20              Enterobacter asburiae  32
21            Enterobacter asburiae_A   6
22            Enterobacter asburiae_B  68
23            Enterobacter asburia

In addition: Warning message:
Expected 8 pieces. Missing pieces filled with `NA` in 1 rows [4957]. 


In [14]:
%%R -o speciesAssemblies

speciesAssemblies <- samples %>%
separate(gtdb_taxonomy, c('f1','f2','f3','f4','f5','f6','f7','f8'), sep = "__") %>%
    mutate(f7 = gsub('_.','',f7)) %>%
    mutate(f7 = gsub('; s','',f7)) %>%
    mutate(f8 = gsub('_.','',f8)) %>%
#	filter(f4 == "Enterobacteriaceae_gtdb") %>%
#	filter(f5 != "other_families") %>%
	count(f7,f8) %>%
	rename(clade = f7, species = f8, assemblies = n)

In addition: Warning message:
Expected 8 pieces. Missing pieces filled with `NA` in 1 rows [4957]. 


In [15]:
%%R
Enterobacteriaceae_best_hits <- hmm_hits %>%
    separate(gtdb_taxonomy, c('f1','f2','f3','f4','f5','f6','f7','f8'), sep = "__") %>%
    mutate(f7 = gsub('_.','',f7)) %>%
    mutate(f7 = gsub('; s','',f7)) %>%
    mutate(f8 = gsub('_.','',f8)) %>%
    rename(clade = f7, species = f8)

head(Enterobacteriaceae_best_hits$species, 20)

 [1] "Escherichia fergusonii" "Escherichia fergusonii" "Escherichia marmotae"  
 [4] "Escherichia coli"       "Escherichia coli"       "Escherichia coli"      
 [7] "Escherichia coli"       "Escherichia coli"       "Escherichia coli"      
[10] "Escherichia coli"       "Escherichia coli"       "Escherichia coli"      
[13] "Escherichia coli"       "Escherichia marmotae"   "Escherichia coli"      
[16] "Salmonella enterica"    "Escherichia coli"       "Escherichia fergusonii"
[19] "Citrobacter koseri"     "Escherichia coli"      


In addition: Warning message:
Expected 8 pieces. Missing pieces filled with `NA` in 1 rows [2462]. 


In [16]:
%%R
Enterobacteriaceae_best_hits %>%
    count(species, name = "n_microcins", )


                      species n_microcins
1    Citrobacter amalonaticus           9
2         Citrobacter braakii           6
3          Citrobacter koseri           2
4       Citrobacter werkmanii           3
5         Citrobacter youngae           2
6     Cronobacter dublinensis           6
7     Cronobacter malonaticus           4
8      Cronobacter muytjensii           2
9       Enterobacter asburiae          40
10  Enterobacter cancerogenus          29
11       Enterobacter cloacae          68
12    Enterobacter hormaechei         516
13         Enterobacter kobei           1


14      Enterobacter ludwigii          61
15          Enterobacter mori           2
16       Escherichia albertii           4
17           Escherichia coli         312
18     Escherichia fergusonii           6
19       Escherichia marmotae           2
20       Klebsiella aerogenes        1846
21   Klebsiella michiganensis         521
22 Klebsiella ornithinolytica         125
23         Klebsiella oxytoca         566
24      Klebsiella planticola          41
25      Klebsiella pneumoniae        3182
26 Klebsiella quasipneumoniae        1914
27       Klebsiella variicola        1800
28          Kosakonia cowanii           1
29           Kosakonia oryzae          26
30    Kosakonia radicincitans          91
31         Kosakonia sacchari          17
32   Leclercia adecarboxylata          63
33    Pluralibacter gergoviae          45
34        Salmonella enterica         110
35                       <NA>           1


In [18]:
%%R
species_microcins <- Enterobacteriaceae_best_hits %>%
	count(species, genome) %>% 
	count(species, name = "n_microcins") %>%
	full_join(speciesAssemblies) %>%
	mutate(percent_microcin = n_microcins / assemblies) %>%
    mutate(percent_microcin = gsub("NA", "0", percent_microcin))

head(species_microcins)

Joining with `by = join_by(species)`
                   species n_microcins       clade assemblies
1 Citrobacter amalonaticus           9 Citrobacter         64
2      Citrobacter braakii           6 Citrobacter        119
3       Citrobacter koseri           2 Citrobacter        108
4    Citrobacter werkmanii           3 Citrobacter         65
5      Citrobacter youngae           2 Citrobacter         40
6  Cronobacter dublinensis           3 Cronobacter         42
    percent_microcin
1           0.140625
2 0.0504201680672269
3 0.0185185185185185
4 0.0461538461538462
5               0.05
6 0.0714285714285714


In [19]:
%%R
species_microcins <- species_microcins %>%
    mutate(species = gsub('Klebsiella planticola','Raoultella planticola',species))%>%
    mutate(species = gsub('Klebsiella ornithinolytica','Raoultella ornithinolytica',species))


In [20]:
%%R
Enterobacteriaciae_species_tree$tip.label <- sub("_", " ", Enterobacteriaciae_species_tree$tip.label)

Enterobacteriaciae_species_tree$tip.label <- ifelse(
  Enterobacteriaciae_species_tree$tip.label %in% species_microcins$species,
  paste0(
    Enterobacteriaciae_species_tree$tip.label,
    " (",
    species_microcins$assemblies[match(Enterobacteriaciae_species_tree$tip.label, species_microcins$species)],
    ")"
  ),
  Enterobacteriaciae_species_tree$tip.label
)

Enterobacteriaciae_species_tree$tip.label


 [1] "Citrobacter werkmanii (65)"       "Citrobacter youngae (40)"        
 [3] "Citrobacter braakii (119)"        "Citrobacter koseri (108)"        
 [5] "Citrobacter amalonaticus (64)"    "Salmonella bongori (32)"         
 [7] "Salmonella enterica (915)"        "Escherichia fergusonii (77)"     
 [9] "Escherichia albertii (106)"       "Escherichia marmotae (82)"       
[11] "Escherichia coli (936)"           "Klebsiella quasipneumoniae (622)"
[13] "Klebsiella pneumoniae (975)"      "Klebsiella variicola (517)"      
[15] "Klebsiella aerogenes (338)"       "Klebsiella oxytoca (173)"        
[17] "Klebsiella michiganensis (268)"   "Raoultella ornithinolytica (91)" 
[19] "Raoultella planticola (43)"       "Pluralibacter gergoviae (22)"    
[21] "Enterobacter cancerogenus (16)"   "Enterobacter asburiae (107)"     
[23] "Lelliottia nimipressuralis (9)"   "Enterobacter kobei (141)"        
[25] "Enterobacter cloacae (240)"       "Enterobacter ludwigii (85)"      
[27] "Enterobacter mori (

In [21]:
%%R
species_microcins$species <- gsub("_"," ",species_microcins$species)
par(family  = "Arial")
# plot(Enterobacteriaciae_species_tree)

In [22]:
%%R -w 600
species_tree_plot <- ggtree(Enterobacteriaciae_species_tree) +
	geom_tiplab(as_ylab=FALSE, fontface=3, size=3)
showtext_auto()
# species_tree_plot

In [23]:
%%R
species_microcins$Clade <- factor(species_microcins$clade, levels = c("Escherichia","Klebsiella","Enterobacter","Kosakonia","Cronobacter"))


In [24]:
%%R
species_microcins %>%
	filter(!(species %in% Enterobacteriaciae_species_tree$tip.label))

#add assemblies to species in species_microcins
species_microcins$species <- paste0(species_microcins$species, " (", species_microcins$assemblies, ")")
	
#first word before space from species into clade
species_microcins$Clade <- gsub(" .*","",species_microcins$species)


In [25]:
%%R
species_microcins

                                  species n_microcins            clade
1           Citrobacter amalonaticus (64)           9      Citrobacter
2               Citrobacter braakii (119)           6      Citrobacter
3                Citrobacter koseri (108)           2      Citrobacter
4              Citrobacter werkmanii (65)           3      Citrobacter
5                Citrobacter youngae (40)           2      Citrobacter
6            Cronobacter dublinensis (42)           3      Cronobacter
7            Cronobacter malonaticus (62)           2      Cronobacter
8              Cronobacter muytjensii (8)           1      Cronobacter
9             Enterobacter asburiae (107)          40     Enterobacter
10         Enterobacter cancerogenus (16)          14     Enterobacter
11             Enterobacter cloacae (240)          66     Enterobacter
12          Enterobacter hormaechei (977)         511     Enterobacter
13               Enterobacter kobei (141)           1     Enterobacter
14    

In [26]:
%%R
species_microcins_hmmer <- species_microcins %>%
	mutate(percent_microcin = n_microcins / assemblies) %>%
    mutate(Clade = gsub('Citrobacter','Escherichia',Clade)) %>%
    mutate(Clade = gsub('Salmonella','Escherichia',Clade)) %>%
    mutate(Clade = gsub('Raoultella','Klebsiella',Clade)) %>%
    mutate(Clade = gsub('Leclercia','Enterobacter',Clade)) %>%
    mutate(Clade = gsub('Lelliottia','Enterobacter',Clade)) %>%
    mutate(Clade = gsub('Atlantibacter','Cronobacter',Clade)) %>%
    mutate(Clade = gsub('Pluralibacter','Klebsiella',Clade))

species_microcins_hmmer$n_microcins[is.na(species_microcins_hmmer$n_microcins)] <- 0
species_microcins_hmmer$percent_microcin[is.na(species_microcins_hmmer$percent_microcin)] <- 0

#count all assemblies, join all
sum(species_microcins_hmmer$assemblies)

[1] 7981


In [27]:
%%R
species_microcins_hmmer$Clade <- factor(species_microcins_hmmer$Clade, levels = c("Escherichia","Klebsiella","Enterobacter","Kosakonia","Cronobacter"))


In [28]:
%%R
species_microcins_hmmer$percent_microcin

# save species_microcins_hmmer as micsearch_plot_data.csv
# write.csv(species_microcins_hmmer, "data/micsearch_plot_data.csv", row.names = FALSE)

 [1] 0.140625000 0.050420168 0.018518519 0.046153846 0.050000000 0.071428571
 [7] 0.032258065 0.125000000 0.373831776 0.875000000 0.275000000 0.523029683
[13] 0.007092199 0.600000000 0.043478261 0.028301887 0.175213675 0.051948052
[19] 0.024390244 1.000000000 1.000000000 0.912087912 1.000000000 0.953488372
[25] 1.000000000 1.000000000 1.000000000 0.055555556 1.000000000 1.000000000
[31] 0.285714286 0.200000000 1.000000000 0.062295082 1.000000000 0.000000000
[37] 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000
[43] 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000
[49] 0.000000000 0.000000000 0.000000000 0.000000000


In [30]:
%%R -w 500

species_tree_microcin_plot <- facet_plot(
    species_tree_plot +
        xlim(0, 100),
        panel = "percent microcin",
        data = species_microcins_hmmer,
        geom = geom_barh,
        mapping = aes(x = percent_microcin*100, fill = Clade),
        stat="identity"
    ) +
    scale_fill_manual(values=c(
        "#7397c7ff", 
        "#8fc960ff", 
        "#dfbf5dff",
        "#cf6e6eff",
        "#a674cbff")) +
    theme_tree2(
        plot.margin = margin(0,0,0,0),
        strip.background = element_blank(),
        strip.text.x = element_blank(),
        #text = element_text(size = 14)
        legend.title = element_text(),
        legend.text = element_text(face = 'italic')
        ) +
    xlab("Percent assemblies with microcins")

species_tree_microcin_plot

filename = paste("plots/fullmicrocinHMM_newsearch.svg")
ggsave(filename, bg = 'transparent', dpi=300)


ℹ invalid tbl_tree object. Missing column: parent,node.
ℹ invalid tbl_tree object. Missing column: parent,node.


Saving 6.94 x 6.67 in image
